In [1]:
import torch
from torch.utils.data import Dataset, random_split
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os.path
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
dev = 'mps' if torch.has_mps else 'cpu'
print(dev)

mps


In [3]:
device = torch.device(dev)

layer_size = 3096

model = nn.Sequential(
    nn.Linear(539, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.ReLU(),
    nn.Linear(layer_size, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.ReLU(),
    nn.Linear(layer_size, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.ReLU(),
    nn.Linear(layer_size, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.ReLU(),
    nn.Linear(layer_size, 1)
)
model.load_state_dict(torch.load('./jun6_11pm.pth'))
model.to(device)
model.eval()

Sequential(
  (0): Linear(in_features=539, out_features=3096, bias=True)
  (1): BatchNorm1d(3096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=3096, out_features=3096, bias=True)
  (4): BatchNorm1d(3096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Linear(in_features=3096, out_features=3096, bias=True)
  (7): BatchNorm1d(3096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Linear(in_features=3096, out_features=3096, bias=True)
  (10): BatchNorm1d(3096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): Linear(in_features=3096, out_features=1, bias=True)
)

In [4]:
test_data = pd.read_csv('./csv_artifacts/test_dataish.csv', dtype=np.float32)
trip_ids = pd.read_csv('./csv_artifacts/test_public.csv')
trip_ids = trip_ids[['TRIP_ID']]
all_xy = test_data.to_numpy()
test_input = torch.tensor(all_xy[:,0:539], dtype=torch.float32).to(device)

In [5]:
print(len(test_input))

320


In [6]:
with torch.no_grad():
    outputs = model(test_input)

In [7]:
outputs = torch.Tensor.cpu(outputs)

In [8]:
output_file = pd.DataFrame(columns=['TRIP_ID', 'TRAVEL_TIME'])
display(output_file)

,TRIP_ID,TRAVEL_TIME


In [9]:
for count, i in enumerate(outputs):
    trip_id = trip_ids.iloc[count].to_string()
    trip_id = trip_id.replace('TRIP_ID', '').strip()
    print(trip_id)
    item = i.item()
    print('Trip ID: {} Result: {}'.format(count+1, item))
    output_file.loc[len(output_file)] = [trip_id, item]


T1
Trip ID: 1 Result: 787.6790161132812
T2
Trip ID: 2 Result: 519.34912109375
T3
Trip ID: 3 Result: 591.7234497070312
T4
Trip ID: 4 Result: 477.83441162109375
T5
Trip ID: 5 Result: 519.9639282226562
T6
Trip ID: 6 Result: 674.95458984375
T7
Trip ID: 7 Result: 708.7317504882812
T8
Trip ID: 8 Result: 433.9061279296875
T9
Trip ID: 9 Result: 820.8461303710938
T10
Trip ID: 10 Result: 745.1779174804688
T11
Trip ID: 11 Result: 465.8665771484375
T12
Trip ID: 12 Result: 893.5962524414062
T13
Trip ID: 13 Result: 530.3740844726562
T14
Trip ID: 14 Result: 833.0613403320312
T15
Trip ID: 15 Result: 806.7117309570312
T16
Trip ID: 16 Result: 748.9382934570312
T17
Trip ID: 17 Result: 718.7669067382812
T18
Trip ID: 18 Result: 893.4020385742188
T19
Trip ID: 19 Result: 679.9800415039062
T20
Trip ID: 20 Result: 757.5242309570312
T21
Trip ID: 21 Result: 693.97265625
T22
Trip ID: 22 Result: 523.971923828125
T23
Trip ID: 23 Result: 417.30657958984375
T24
Trip ID: 24 Result: 960.4342651367188
T25
Trip ID: 25 Re

In [10]:
display(output_file)

,TRIP_ID,TRAVEL_TIME
0,T1,787.679016
1,T2,519.349121
2,T3,591.723450
3,T4,477.834412
4,T5,519.963928
...,...,...
315,T323,349.685242
316,T324,574.999451
317,T325,1871.852173
318,T326,360.334412


In [11]:
output_file.to_csv('submissions.csv', index=False)